In [24]:
import glob

# 目标文件名
output_file = "merged_file.txt"

# 获取所有以 new_file_ 开头的 .txt 文件
file_list = glob.glob("new_file_*.txt")

# 初始化一个空列表来存储所有的 JSON 对象
merged_data = []

# 遍历每个文件并将内容合并到 merged_data 中
for file_name in file_list:
    with open(file_name, "r", encoding="utf-8") as file:
        content = file.read().strip()
        # 去掉开头的 '''json[ 和结尾的 ]'''
        if content.startswith("```json\n["):
            content = content[7:].strip()  # 去掉 '''json[
        if content.endswith("]\n```"):
            content = content[:-4].strip()  # 去掉 ]'''
        elif content.endswith("]"):
            content = content[:-1].strip()  # 去掉 ]

        # 确保去掉多余的大括号
        if content.startswith("["):
            content = content[1:].strip()
        if content.endswith("]"):
            content = content[:-1].strip()

        # 将每个文件的内容按 }, { 分割，并合并到 merged_data 列表中
        json_objects = content.split("}, {")
        for i in range(len(json_objects)):
            if i == 0:
                if not json_objects[i].startswith("{"):
                    json_objects[i] = "{" + json_objects[i]
            if i == len(json_objects) - 1:
                if not json_objects[i].endswith("}"):
                    json_objects[i] = json_objects[i] + "}"
            else:
                json_objects[i] = "{" + json_objects[i] + "}"
        merged_data.extend(json_objects)

# 将合并后的数据写入到目标文件中
with open(output_file, "w", encoding="utf-8") as output:
    output.write("[\n")
    output.write(",\n".join(merged_data))
    output.write("\n]")

print(f"所有文件已成功合并到 {output_file}")


所有文件已成功合并到 merged_file.txt


In [ ]:
import pandas as pd
import numpy as np
import re

# 假设df是你的DataFrame
df = pd.DataFrame({
    'match_result': [
        # ... 你的数据，每个元素都是一个字典，例如：
        {'id': 'as-1ihuazbnqy', 'object': 'chat.completion', 'created': 1720164118, 
         'result': 'accuracy: 0.1\n\n解释：...'},
        # ... 其他行
    ]
})

# 自定义一个函数来解析'result'字段并提取accuracy值
def extract_accuracy(match_result_dict):
    result_str = match_result_dict.get('result', '')
    # 使用正则表达式匹配各种格式的冒号后的数字
    match = re.search(r'accuracy[\s:：]+([0-9.]+)', result_str)
    if match:
        return float(match.group(1))  # 将匹配到的字符串转换为浮点数
    else:
        return np.nan  # 如果没有找到匹配项，返回np.nan

# 应用自定义函数到'match_result'列，并创建一个新列'gpt_accuracy'
df['gpt_accuracy'] = df['match_result'].apply(extract_accuracy)

# 显示结果
print(df)